# Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [2]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o" # After try MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

C:\Users\Usuario\Documents\VSCode Projects\python\intro-to-langsmith\intro-to-ls\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-fd161790' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=6a8214b5-1c39-403e-a8d0-49c7127a8929




15it [00:35,  2.38s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What types of evaluations can be performed usi...,LangSmith supports evaluations such as complet...,None,LangSmith supports a variety of evaluation typ...,1,3.944472,1d57fa4d-de29-49ad-8422-deb5a31b39be,5e535325-4e26-427f-8f1c-0208d02ee314
1,What is LangSmith used for in three sentences?,"LangSmith is a platform designed for building,...",None,LangSmith is a platform designed for the devel...,1,2.536665,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,8fec744b-a8be-4beb-8791-019e3eb5ba7c
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.436547,5695c832-c42d-4c8f-acdb-a17f38b1600c,7cb53edd-b3e6-44a8-b4d2-8c393156fbd7
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,6.197020,f2adbfb8-2856-4481-97c9-0d7902074cdd,550d87e7-b26e-44b3-9510-10b51c42d65e
4,How can I integrate LangSmith with my existing...,You can integrate LangSmith with your existing...,None,Integrating LangSmith with existing workflows ...,1,1.980316,4c197cd8-2f94-485c-8e20-47dcd451f1b9,c54503d5-6795-4824-bc34-bd64b7ec3d35
5,What are the advantages of using LangSmith for...,The advantages of using LangSmith for LLM appl...,None,"It provides several advantages, including enha...",1,2.104347,e30596e5-1ff0-4a1b-a4b7-3ec203e7353e,f2707782-369f-491e-9c32-d92c0983de38
6,Is there a Javascript LangSmith SDK?,"Yes, there is a JavaScript LangSmith SDK.",None,"Yes, there is a Javascript LangSmith SDK!",1,0.834972,cdcc011d-fa6c-4eb6-93c1-4ba4ac9f211c,03d66b65-e421-4189-b474-e893f76fa813
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.831310,9e04a873-d592-4c97-bd46-ff510efdca22,8fbb4964-7ba3-46ec-b652-23739aa657d4
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.473776,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,7a006ad9-0963-4964-a799-e6d11b20cbca
9,Can LangSmith be used for finetuning and model...,The provided context does not mention finetuni...,None,"Yes, LangSmith can be used for fine-tuning and...",1,3.829818,57df8ff0-bf0d-4494-a51f-d84017fc121b,90bb6bdf-837d-42c8-a8fd-a074634a2416


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [ ]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [17]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits="base"),   # We use param as_of="initial dataset" to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-4f9eedd4' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=261c8c58-afd3-421b-b755-0c6b5976800e




15it [00:25,  1.70s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What types of evaluations can be performed usi...,LangSmith supports several types of evaluation...,None,LangSmith supports a variety of evaluation typ...,1,2.564661,1d57fa4d-de29-49ad-8422-deb5a31b39be,c7fe6b4b-037f-4c9e-87aa-35d840c07d66
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for the devel...,None,LangSmith is a platform designed for the devel...,1,1.464699,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,c63cb842-9775-49b3-add0-fcb81b192816
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.735839,5695c832-c42d-4c8f-acdb-a17f38b1600c,bf74da73-5c6a-446b-bd79-229f83b0b9e8
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,1,1.895934,f2adbfb8-2856-4481-97c9-0d7902074cdd,b0f217d2-81c4-4b13-be87-9eccf1ec7710
4,How can I integrate LangSmith with my existing...,You can integrate LangSmith with your existing...,None,Integrating LangSmith with existing workflows ...,1,1.518970,4c197cd8-2f94-485c-8e20-47dcd451f1b9,ec465b13-8a65-4834-8004-214e1bc0ea70
5,What are the advantages of using LangSmith for...,LangSmith offers several advantages for LLM ap...,None,"It provides several advantages, including enha...",1,1.583093,e30596e5-1ff0-4a1b-a4b7-3ec203e7353e,02fa4ef7-b0c6-4dc4-a24c-fcdc8d2a53fe
6,Is there a Javascript LangSmith SDK?,"Yes, there is a LangSmith SDK available for Ja...",None,"Yes, there is a Javascript LangSmith SDK!",0,1.267351,cdcc011d-fa6c-4eb6-93c1-4ba4ac9f211c,3d87ba58-b31f-481e-b58d-d47e37cd1ca0
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.139871,9e04a873-d592-4c97-bd46-ff510efdca22,021535ca-80b9-4736-952b-270859f29a32
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.843081,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,43aafd24-86b2-4c70-9301-4d93bdb761de
9,Can LangSmith be used for finetuning and model...,The provided documentation does not mention La...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.331433,57df8ff0-bf0d-4494-a51f-d84017fc121b,6ee30348-29af-4e50-831d-ecebca8c922a


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [8]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-aced68c8' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=46e7f910-f952-4dfb-87d9-1050e62a1d1f




4it [00:08,  2.03s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What types of evaluations can be performed usi...,LangSmith supports evaluations such as complet...,None,LangSmith supports a variety of evaluation typ...,1,2.139769,1d57fa4d-de29-49ad-8422-deb5a31b39be,da624e18-575a-479a-8098-835fe0ef1ca4
1,What is LangSmith used for in three sentences?,"LangSmith is used for developing, monitoring, ...",None,LangSmith is a platform designed for the devel...,1,2.213721,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,2ea9566f-9a01-4639-9158-f594f6476463
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.660626,5695c832-c42d-4c8f-acdb-a17f38b1600c,3a669df0-07f2-4f73-9825-d7215d994e16
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,1.770756,f2adbfb8-2856-4481-97c9-0d7902074cdd,1f56c527-331b-4a11-abf5-922bfcc89418


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [19]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "57df8ff0-bf0d-4494-a51f-d84017fc121b",
            "97c84fa0-3a16-4fc1-bd39-52e6ac3f819e"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-3d93911c' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=9c7433b4-9e95-46b9-b5f5-8c69ef7655df




2it [00:03,  1.89s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used for finetuning and model...,"No, LangSmith is not designed for finetuning a...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.855574,57df8ff0-bf0d-4494-a51f-d84017fc121b,9f8f6136-caf4-4c5c-b0fa-156d05e2017d
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.436417,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,ded99f11-c134-4e4d-9fab-737896122720


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [20]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-4a70617c' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=1863dc0f-e6dd-4245-94a3-e305bcefadc7




30it [00:59,  2.00s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What types of evaluations can be performed usi...,LangSmith supports evaluations such as complet...,None,LangSmith supports a variety of evaluation typ...,1,5.183898,1d57fa4d-de29-49ad-8422-deb5a31b39be,8c3c5892-ee57-4e2c-be8f-67d140d4202c
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for developin...,None,LangSmith is a platform designed for the devel...,1,2.350614,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,08b9f74e-38b7-4f34-92d3-a8311f551e46
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.420174,5695c832-c42d-4c8f-acdb-a17f38b1600c,9e2d51f6-099f-438c-8a21-d1398992baa0
3,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith for LangChain, ...",None,To set up tracing to LangSmith while using Lan...,1,1.888865,f2adbfb8-2856-4481-97c9-0d7902074cdd,af56cdb6-5417-4077-a3ad-9c68f5ee56df
4,How can I integrate LangSmith with my existing...,To integrate LangSmith with your existing work...,None,Integrating LangSmith with existing workflows ...,1,1.880060,4c197cd8-2f94-485c-8e20-47dcd451f1b9,dac36866-c9ec-4af9-a77f-819c46068504
5,What are the advantages of using LangSmith for...,LangSmith provides several advantages for deve...,None,"It provides several advantages, including enha...",1,2.880567,e30596e5-1ff0-4a1b-a4b7-3ec203e7353e,d587149f-db4d-4a53-8156-63129bee7430
6,Is there a Javascript LangSmith SDK?,"Yes, there is a JavaScript/TypeScript LangSmit...",None,"Yes, there is a Javascript LangSmith SDK!",0,0.852876,cdcc011d-fa6c-4eb6-93c1-4ba4ac9f211c,519d3da3-a4fc-4db5-8aa8-54d8e1651479
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.806766,9e04a873-d592-4c97-bd46-ff510efdca22,7db37e69-b511-40b7-87a1-d2f40c74e164
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.633821,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,6db3ebc9-dcdc-4213-867e-f1c9bd4c10ba
9,Can LangSmith be used for finetuning and model...,The provided context does not mention the capa...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.441473,57df8ff0-bf0d-4494-a51f-d84017fc121b,54e2823f-cc70-4831-a466-956d58dc733c


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [21]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-8c349c34' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=5c6b0977-a3d9-4991-b3f3-4fe20600c8a5




15it [00:11,  1.33it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is LangSmith used for in three sentences?,LangSmith is a platform designed for developin...,None,LangSmith is a platform designed for the devel...,1,1.871481,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,1938661a-d313-4e13-89bc-c9592d443173
1,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.964260,5695c832-c42d-4c8f-acdb-a17f38b1600c,a2baaaee-3be8-4185-93d9-c0d8044b40f0
2,What types of evaluations can be performed usi...,LangSmith provides strong and configurable off...,None,LangSmith supports a variety of evaluation typ...,1,1.986790,1d57fa4d-de29-49ad-8422-deb5a31b39be,5b675710-00df-41c6-b368-63d60230973c
3,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,1,1.674036,f2adbfb8-2856-4481-97c9-0d7902074cdd,32f1dd7d-8c9a-42e4-a3e3-6ea9f3f1686e
4,How can I integrate LangSmith with my existing...,You can integrate LangSmith with your existing...,None,Integrating LangSmith with existing workflows ...,1,1.755727,4c197cd8-2f94-485c-8e20-47dcd451f1b9,d46d2928-a4fe-4f3f-aaab-c54217276043
5,Is there a Javascript LangSmith SDK?,"Yes, there is a JavaScript/TypeScript LangSmit...",None,"Yes, there is a Javascript LangSmith SDK!",1,1.464119,cdcc011d-fa6c-4eb6-93c1-4ba4ac9f211c,09cb7b07-f28a-44f6-b9dd-b6cb3525992e
6,What are the advantages of using LangSmith for...,LangSmith offers several advantages for develo...,None,"It provides several advantages, including enha...",1,3.508374,e30596e5-1ff0-4a1b-a4b7-3ec203e7353e,5367ebcc-7f36-4fd1-954e-6d668232c874
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.363058,9e04a873-d592-4c97-bd46-ff510efdca22,00838d07-1ecd-41e4-bfe4-6b407a84be08
8,Can LangSmith be used for finetuning and model...,"No, LangSmith focuses on monitoring, evaluatin...",None,"Yes, LangSmith can be used for fine-tuning and...",1,0.982125,57df8ff0-bf0d-4494-a51f-d84017fc121b,2d22b05a-54bf-410b-bc1f-463dd5b4364c
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.622915,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,a65cd38f-58e2-48e2-bcb3-770cf9654b05


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [22]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-f8a8666b' at:
https://smith.langchain.com/o/036832fd-8ebc-5afd-896b-d2f686653c0c/datasets/cfe59771-a121-4e2a-9a6e-9654a79ca10a/compare?selectedSessions=8005f16e-00b5-48ee-a9e2-2126cd01e976




15it [00:27,  1.81s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What types of evaluations can be performed usi...,LangSmith allows for various types of evaluati...,None,LangSmith supports a variety of evaluation typ...,1,2.538405,1d57fa4d-de29-49ad-8422-deb5a31b39be,0ebe0d3f-a70b-427c-8798-b581e978a6ab
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for LLM appli...,None,LangSmith is a platform designed for the devel...,1,1.323875,2335fe29-16d2-40fb-9744-d2e2f1ac6a53,d1c70837-23d9-4109-aeef-8643946e489f
2,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.979104,5695c832-c42d-4c8f-acdb-a17f38b1600c,0f1188aa-d6f2-4b4f-8853-caeac645e033
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,4.077965,f2adbfb8-2856-4481-97c9-0d7902074cdd,7b17560e-d3d1-4502-af19-26119dc30b9a
4,How can I integrate LangSmith with my existing...,You can integrate LangSmith with your existing...,None,Integrating LangSmith with existing workflows ...,1,1.536458,4c197cd8-2f94-485c-8e20-47dcd451f1b9,72357ed2-1f4f-495b-b5d1-15c87830958e
5,What are the advantages of using LangSmith for...,The advantages of using LangSmith for LLM appl...,None,"It provides several advantages, including enha...",1,1.472105,e30596e5-1ff0-4a1b-a4b7-3ec203e7353e,96b476ef-404e-474e-854b-89a50102c4e7
6,Is there a Javascript LangSmith SDK?,"Yes, there is a JavaScript (JS/TS) LangSmith S...",None,"Yes, there is a Javascript LangSmith SDK!",1,0.952202,cdcc011d-fa6c-4eb6-93c1-4ba4ac9f211c,6131d5f8-ba71-4e4c-9f2c-e5e7eb7cebc3
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.116573,9e04a873-d592-4c97-bd46-ff510efdca22,022181f2-23f9-4447-bc6b-365cb029a9e7
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.220220,97c84fa0-3a16-4fc1-bd39-52e6ac3f819e,2b9ec926-32d7-4ba6-8c71-ef1bc12808fd
9,Can LangSmith be used for finetuning and model...,The provided context does not mention LangSmit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.588908,57df8ff0-bf0d-4494-a51f-d84017fc121b,d4e2107e-6fa4-4547-98be-cc203109a7f0


# Takeaways

**Experiments**

running your application over a dataset, and evaluating its performance.

experiments can be run using the langSmith SDK with __evaluate()__

experiments can be run over an:

- entire dataset
- a specific version
- a split
- specific examples

experiments can be run with other parameters

- repetitions
- concurrent threads
- metadata